In [ ]:
// Audio Splitter for Deno - Handles M4A files
// This script splits audio files into chunks based on silence detection

// Import Deno standard library functions

import { basename, extname, join} from "@std/path";

/**
 * Interface representing an audio chunk with start, end, and duration
 */
interface AudioChunk {
  start: number;
  end: number;
  duration: number;
}

/**
 * Split an M4A file into chunks based on silence detection
 * @param inputFile Path to the input M4A file
 * @param outputDir Directory to save the output chunks
 * @param maxChunkDuration Maximum duration of each chunk in seconds (default: 180 seconds = 3 minutes)
 * @param silenceDuration Minimum silence duration to split at in seconds (default: 5 seconds)
 * @param silenceThreshold dB threshold for silence detection (default: -30dB)
 */
async function splitAudioFile(
  inputFile: string,
  outputDir: string,
  maxChunkDuration: number = 180, // 3 minutes in seconds
  silenceDuration: number = 1, // 1 second of silence
  silenceThreshold: number = -30 // -30dB threshold for silence
): Promise<void> {
  // Create output directory if it doesn't exist
  // await ensureDir(outputDir);

  const inputFileName = basename(inputFile, extname(inputFile));

  // Step 1: Analyze the audio file to detect silence
  console.log("Analyzing audio for silence...");
  const silenceDetectionCmd = new Deno.Command("ffmpeg", {
    args: [
      "-i",
      inputFile,
      "-af",
      `silencedetect=noise=${silenceThreshold}dB:d=${silenceDuration}`,
      "-f",
      "null",
      "-",
    ],
    stderr: "piped",
  });

  // ffmpeg -i Long_Audio_File.mp3 -af silencedetect=d=0.1 -f null - |& awk '/silence_end/ {print $4,$5}' | awk '{S=$2;printf "%d:%02d:%02d\n",S/(60*60),S%(60*60)/60,S%60}'

  try {
    const { stderr } = await silenceDetectionCmd.output();
    const stderrText = new TextDecoder().decode(stderr);

    // Parse the silence detection output
    const silenceStarts: number[] = [];
    const silenceEnds: number[] = [];

    // Extract silence start and end times
    const startMatches = stderrText.matchAll(/silence_start: ([\d.]+)/g);
    const endMatches = stderrText.matchAll(/silence_end: ([\d.]+)/g);

    for (const match of startMatches) {
      silenceStarts.push(parseFloat(match[1]));
    }

    for (const match of endMatches) {
      silenceEnds.push(parseFloat(match[1]));
    }

    // Get the duration of the audio file
    const durationMatch = stderrText.match(/Duration: (\d+):(\d+):(\d+.\d+)/);
    let totalDuration = 0;

    if (durationMatch) {
      const hours = parseInt(durationMatch[1]);
      const minutes = parseInt(durationMatch[2]);
      const seconds = parseFloat(durationMatch[3]);

      totalDuration = hours * 3600 + minutes * 60 + seconds;
    } else {
      throw new Error("Could not determine audio file duration");
    }

    // Step 2: Determine chunk boundaries
    const chunks: AudioChunk[] = [];
    let currentStart = 0;

    // Process the silence periods to create chunks
    for (let i = 0; i < silenceStarts.length; i++) {
      const silenceStart = silenceStarts[i];
      const silenceEnd = silenceEnds[i];

      // If the silence is too close to the start of the audio, skip it
      if (silenceStart < 10) {
        continue;
      }

      // Check if we've reached our max chunk duration
      if (silenceStart - currentStart >= maxChunkDuration) {
        chunks.push({
          start: currentStart,
          end: silenceStart,
          duration: silenceStart - currentStart,
        });
        currentStart = silenceEnd;
      }
    }

    // Add the final chunk if needed
    if (currentStart < totalDuration) {
      chunks.push({
        start: currentStart,
        end: totalDuration,
        duration: totalDuration - currentStart,
      });
    }

    // Step 3: Split the audio file into chunks
    console.log(`Splitting audio into ${chunks.length} chunks...`);

    // Process each chunk
    for (let i = 0; i < chunks.length; i++) {
      const chunk = chunks[i];
      const outputFile = join(outputDir, `${inputFileName}_chunk${i + 1}.m4a`);

      const ffmpegProcess = new Deno.Command("ffmpeg", {
        args: [
          "-i",
          inputFile,
          "-ss",
          chunk.start.toString(),
          "-t",
          chunk.duration.toString(),
          "-c",
          "copy", // Copy codec for fast processing without re-encoding
          outputFile,
        ],
        stderr: "piped",
      });

      const status = await ffmpegProcess.output();

      if (status.code === 0) {
        console.log(
          `Created chunk ${i + 1}: ${outputFile} (${Math.round(
            chunk.duration
          )} seconds)`
        );
      } else {
        const errorText = new TextDecoder().decode(status.stderr);
        console.error(`Error creating chunk ${i + 1}: ${errorText}`);
        throw new Error(`Failed to create chunk ${i + 1}`);
      }
    }

    console.log("Audio splitting completed successfully!");
  } catch (error) {
    console.error("Error splitting audio:", error);
    throw error;
  }
}

/**
 * Main function to run the script
 */
async function main() {
  // Use Deno.args to get command line arguments
  const inputFile = "./pod.m4a";
  const outputDir = "../data/episode1";

  if (!inputFile) {
    console.error("Please provide an input M4A file path as argument");
    Deno.exit(1);
  }

  try {
    await splitAudioFile(inputFile, outputDir);
  } catch (error) {
    console.error("Failed to split audio:", error);
    Deno.exit(1);
  }
}

await main();

Analyzing audio for silence...
Splitting audio into 11 chunks...
Created chunk 1: pod_chunk1.m4a (188 seconds)
Created chunk 2: pod_chunk2.m4a (475 seconds)
Created chunk 3: pod_chunk3.m4a (184 seconds)
Created chunk 4: pod_chunk4.m4a (547 seconds)
Created chunk 5: pod_chunk5.m4a (320 seconds)
Created chunk 6: pod_chunk6.m4a (186 seconds)
Created chunk 7: pod_chunk7.m4a (282 seconds)
Created chunk 8: pod_chunk8.m4a (198 seconds)
Created chunk 9: pod_chunk9.m4a (256 seconds)
Created chunk 10: pod_chunk10.m4a (289 seconds)
Created chunk 11: pod_chunk11.m4a (208 seconds)
Audio splitting completed successfully!
